In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import uproot as ur
import glob
import os
import shutil
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# CSV Creation
Instead of saving the data on root files I just saved them to the csv files.
Now I one-hot encode the particle id's and add the weights and chw values

In [7]:
CEDAR_PRE_PATH = "/home/aaa327/projects/ctb-stelzer/aaa327"
LOCAL_PRE_PATH = "/Users/atakanazakli/SFU_Thesis_Work"

SEARCH_DIR = f"{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features"


def find_files(search_dir=".", file_name_to_be_searched=None):
    search_pattern = os.path.join(search_dir, "**", file_name_to_be_searched)
    return glob.glob(search_pattern, recursive=True)


FILE_PATHS = find_files(search_dir=SEARCH_DIR, file_name_to_be_searched='*.csv')
FILE_PATHS

['/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-01.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_1.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-10.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-1.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_sm.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_01.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_2.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-5.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-2.root.csv',
 '/Users/atakanazakli/SFU_Thesis

In [3]:
FILE_PATHS = [f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_sm.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_-01.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_-1.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_-2.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_-5.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_-10.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_01.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_1.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_2.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_5.root.csv',
 f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/output_bsm_10.root.csv',
 ]
FILE_PATHS

['/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_sm.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-01.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-1.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-2.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-5.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_-10.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_01.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_1.root.csv',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/output_bsm_2.root.csv',
 '/Users/atakanazakli/SFU_Thesis

In [5]:
weight_list = [
['cHW0.000', 0.00112598, 0.0],
['cHW-0.10', 0.001147274, -0.1],
['cHW-1.00', 0.0014416, -1.0],
['cHW-2.00', 0.0020291, -2.0],
['cHW-5.00', 0.00608081, -5.0],
['cHW-10.0', 0.025423099999999997, -10.0],
['cHW0.100', 0.0011079080000000001, 0.1],
['cHW1.000', 0.0010254490000000001, 1.0],
['cHW2.000', 0.001091368, 2.0],
['cHW5.000', 0.00187389, 5.0],
['cHW10.00', 0.00371504, 10.0],
]

In [6]:
def encode_class_columns(df):
    """
    One-hot encodes the l0_id, l1_id (e-/e+) and q0_id, q1_id (dq/uq) columns
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with l0_id, l1_id, q0_id, q1_id columns
    
    Returns:
    pandas.DataFrame: DataFrame with columns one-hot encoded
    """
    # Create a copy of the DataFrame
    df_encoded = df.copy()
    
    # Encode l0_id and l1_id (11/-11 columns)
    for col in ['l0_id', 'l1_id']:
        df_encoded[f'{col}_e-'] = (df[col] == 11).astype(int)
        df_encoded[f'{col}_e+'] = (df[col] == -11).astype(int)
        
    # Encode q0_id and q1_id (1/2 columns)
    for col in ['q0_id', 'q1_id']:
        df_encoded[f'{col}_dq'] = (df[col] == 1).astype(int)
        df_encoded[f'{col}_uq'] = (df[col] == 2).astype(int)
    
    # Drop original columns
    df_encoded = df_encoded.drop(columns=['l0_id', 'l1_id', 'q0_id', 'q1_id'])
    
    return df_encoded

# Example usage:

# Your data
example_data = {
    'l0_id': [-11, 11, 11, 11, 11],
    'l1_id': [11, -11, -11, -11, -11],
    'q0_id': [1, 2, 2, 1, 1],
    'q1_id': [2, 1, 1, 2, 2]
}
example_df = pd.DataFrame(example_data)

# Apply encoding
example_df_encoded = encode_class_columns(example_df)
example_df_encoded.head()

,l0_id_e-,l0_id_e+,l1_id_e-,l1_id_e+,q0_id_dq,q0_id_uq,q1_id_dq,q1_id_uq
0,0,1,1,0,1,0,0,1
1,1,0,0,1,0,1,1,0
2,1,0,0,1,0,1,1,0
3,1,0,0,1,1,0,0,1
4,1,0,0,1,1,0,0,1


In [18]:
test_df = pd.read_csv(FILE_PATHS[0])
test_df.head()

,l0_id,l1_id,q0_id,q1_id,l0_pt,l1_pt,q0_pt,q1_pt,l0_phi,l1_phi,q0_phi,q1_phi,l0_eta,l1_eta,q0_eta,q1_eta,l0_m,l1_m,q0_m,q1_m,l0_e,l1_e,q0_e,q1_e,met_et,met_phi,m_ll,m_qq,pt_ll,pt_qq,d_phi_ll,d_phi_qq,d_eta_ll,d_eta_qq,d_y_ll,d_y_qq,sqrtHT,MET_sig,m_l0q0,m_l0q1,m_l1q0,m_l1q1
0,11,-11,2,1,114.490093,42.362408,252.303723,31.771265,-2.354777,-1.332412,0.683512,3.075705,-1.327065,-1.488743,-2.031290,2.792608,-0.001031,0.000245,-0.003391,0.001526,230.996226,98.644673,978.317857,260.282905,100.998772,-2.743096,69.065507,1004.682902,141.278871,230.064283,-1.022365,-2.392193,0.161678,-4.823898,0.161678,-4.823898,23.279310,4.338564,360.783424,468.103452,183.860152,313.345669
1,-11,11,2,1,14.149076,8.927327,77.288809,56.531621,-0.431505,2.710642,2.792856,1.100597,3.263323,1.380969,-3.088486,2.372211,-0.001671,0.000148,0.002845,0.001935,185.172529,18.881695,849.767215,305.676230,85.765897,-1.055257,33.190396,1014.394249,5.221753,90.058769,3.141038,1.692260,1.882354,-5.460697,1.882354,-5.460697,15.577636,5.505707,793.330434,47.076272,242.640364,39.834522
2,11,-11,2,1,39.008558,9.447233,53.517911,45.112543,-0.772832,-1.588745,-1.351145,1.536884,0.136348,0.610233,-2.936717,4.250636,0.000140,-0.000070,0.002899,-0.006027,39.371721,11.261500,505.928428,1582.644088,59.978958,2.322270,17.785877,1788.256374,45.999397,15.001616,0.815913,-2.888029,-0.473885,-7.187353,-0.473885,-7.187353,14.389760,4.168169,204.231266,331.832043,128.762006,130.783442
3,11,-11,2,1,62.168410,31.144976,111.580677,52.043403,0.781485,0.513821,2.704831,-1.963006,2.335536,1.909460,-3.010168,3.131070,-0.000674,0.001113,-0.009419,-0.002325,324.262793,107.412830,1134.780921,596.993418,76.564319,-0.863603,22.242951,1642.755358,92.571564,121.002054,0.267664,-1.615349,0.426076,-6.141238,0.426076,-6.141238,18.262031,4.192541,1208.100598,120.814845,692.817691,92.349083
4,11,-11,2,1,51.878371,31.590252,83.984114,64.091713,0.331658,0.002331,-2.162397,2.160402,0.114070,0.821951,-2.420317,2.930326,0.000119,-0.000098,0.001038,0.001313,52.216255,42.875983,476.119309,602.049256,1.906682,-0.279127,32.128412,1066.996636,82.406723,84.096916,0.329327,1.960386,-0.707881,-5.350643,-0.707881,-5.350643,15.279108,0.124790,249.461615,239.727874,266.401816,138.428561


In [19]:
test_df_encoded = encode_class_columns(test_df)
test_df_encoded.head()

,l0_pt,l1_pt,q0_pt,q1_pt,l0_phi,l1_phi,q0_phi,q1_phi,l0_eta,l1_eta,q0_eta,q1_eta,l0_m,l1_m,q0_m,q1_m,l0_e,l1_e,q0_e,q1_e,met_et,met_phi,m_ll,m_qq,pt_ll,pt_qq,d_phi_ll,d_phi_qq,d_eta_ll,d_eta_qq,d_y_ll,d_y_qq,sqrtHT,MET_sig,m_l0q0,m_l0q1,m_l1q0,m_l1q1,l0_id_e-,l0_id_e+,l1_id_e-,l1_id_e+,q0_id_dq,q0_id_uq,q1_id_dq,q1_id_uq
0,114.490093,42.362408,252.303723,31.771265,-2.354777,-1.332412,0.683512,3.075705,-1.327065,-1.488743,-2.031290,2.792608,-0.001031,0.000245,-0.003391,0.001526,230.996226,98.644673,978.317857,260.282905,100.998772,-2.743096,69.065507,1004.682902,141.278871,230.064283,-1.022365,-2.392193,0.161678,-4.823898,0.161678,-4.823898,23.279310,4.338564,360.783424,468.103452,183.860152,313.345669,1,0,0,1,0,1,1,0
1,14.149076,8.927327,77.288809,56.531621,-0.431505,2.710642,2.792856,1.100597,3.263323,1.380969,-3.088486,2.372211,-0.001671,0.000148,0.002845,0.001935,185.172529,18.881695,849.767215,305.676230,85.765897,-1.055257,33.190396,1014.394249,5.221753,90.058769,3.141038,1.692260,1.882354,-5.460697,1.882354,-5.460697,15.577636,5.505707,793.330434,47.076272,242.640364,39.834522,0,1,1,0,0,1,1,0
2,39.008558,9.447233,53.517911,45.112543,-0.772832,-1.588745,-1.351145,1.536884,0.136348,0.610233,-2.936717,4.250636,0.000140,-0.000070,0.002899,-0.006027,39.371721,11.261500,505.928428,1582.644088,59.978958,2.322270,17.785877,1788.256374,45.999397,15.001616,0.815913,-2.888029,-0.473885,-7.187353,-0.473885,-7.187353,14.389760,4.168169,204.231266,331.832043,128.762006,130.783442,1,0,0,1,0,1,1,0
3,62.168410,31.144976,111.580677,52.043403,0.781485,0.513821,2.704831,-1.963006,2.335536,1.909460,-3.010168,3.131070,-0.000674,0.001113,-0.009419,-0.002325,324.262793,107.412830,1134.780921,596.993418,76.564319,-0.863603,22.242951,1642.755358,92.571564,121.002054,0.267664,-1.615349,0.426076,-6.141238,0.426076,-6.141238,18.262031,4.192541,1208.100598,120.814845,692.817691,92.349083,1,0,0,1,0,1,1,0
4,51.878371,31.590252,83.984114,64.091713,0.331658,0.002331,-2.162397,2.160402,0.114070,0.821951,-2.420317,2.930326,0.000119,-0.000098,0.001038,0.001313,52.216255,42.875983,476.119309,602.049256,1.906682,-0.279127,32.128412,1066.996636,82.406723,84.096916,0.329327,1.960386,-0.707881,-5.350643,-0.707881,-5.350643,15.279108,0.124790,249.461615,239.727874,266.401816,138.428561,1,0,0,1,0,1,1,0


In [20]:
del test_df_encoded, test_df

In [21]:
for file_path in FILE_PATHS:
    print(file_path.split("/")[-1].split(".")[0] + ".csv")

output_sm.csv
output_bsm_-01.csv
output_bsm_-1.csv
output_bsm_-2.csv
output_bsm_-5.csv
output_bsm_-10.csv
output_bsm_01.csv
output_bsm_1.csv
output_bsm_2.csv
output_bsm_5.csv
output_bsm_10.csv


In [ ]:
for i, file_path in enumerate(FILE_PATHS):
    pd_df = pd.read_csv(file_path)
    print(file_path[87:105].split(".")[0], ' vs ', weight_list[i][0])
    print(file_path)
    print(f'cHW value: {weight_list[i][2]}')
    print(f'Weight of : {weight_list[i][0]} is {weight_list[i][1]} \n')
    # pd_version = root_file['test;2'].arrays(library='pd')
    pd_df_encoded = encode_class_columns(pd_df)
    pd_df_encoded["weight"] = weight_list[i][1]
    pd_df_encoded["cHW"] = weight_list[i][2]
    
    pd_save_dir = file_path.split("/")[-1].split(".")[0] + ".csv"
    pd_df_encoded.to_csv(pd_save_dir, index=False)

    file_path = None
    pd_df = None
    # root_file = None
    pd_df_encoded = None
    pd_save_dir = None

# All data csv

In [23]:
df_sm = pd.read_csv('output_sm.csv',)
df_1 = pd.read_csv('output_bsm_-0.1.csv',)
df_2 = pd.read_csv('output_bsm_-1.csv',)
df_3 = pd.read_csv('output_bsm_-2.csv',)
df_4 = pd.read_csv('output_bsm_-5.csv',)
df_5 = pd.read_csv('output_bsm_-10.csv',)
df_6 = pd.read_csv('output_bsm_0.1.csv',)
df_7 = pd.read_csv('output_bsm_1.csv',)
df_8 = pd.read_csv('output_bsm_2.csv',)
df_9 = pd.read_csv('output_bsm_5.csv')
df_10 = pd.read_csv('output_bsm_10.csv',)

In [24]:
str(df_1['weight'].copy().to_numpy()[0])

'0.001147274'

In [25]:
str(df_2['weight'].copy().to_numpy()[0])

'0.0014416'

In [26]:
df_list = [df_sm.copy(), df_1.copy(), df_2.copy(), df_3.copy(), df_4.copy(), df_5.copy(), df_6.copy(), df_7.copy(), df_8.copy(), df_9.copy(), df_10.copy()]

In [27]:
for df in df_list:
    print(str(df['weight'].copy().to_numpy()[0]))

0.00112598
0.001147274
0.0014416
0.0020291
0.00608081
0.0254230999999999
0.001107908
0.001025449
0.001091368
0.00187389
0.00371504


In [28]:
all_dataset_df = pd.concat([df_sm, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10], ignore_index=True)
all_dataset_df.head()

,l0_pt,l1_pt,q0_pt,q1_pt,l0_phi,l1_phi,q0_phi,q1_phi,l0_eta,l1_eta,q0_eta,q1_eta,l0_m,l1_m,q0_m,q1_m,l0_e,l1_e,q0_e,q1_e,met_et,met_phi,m_ll,m_qq,pt_ll,pt_qq,d_phi_ll,d_phi_qq,d_eta_ll,d_eta_qq,d_y_ll,d_y_qq,sqrtHT,MET_sig,m_l0q0,m_l0q1,m_l1q0,m_l1q1,l0_id_e-,l0_id_e+,l1_id_e-,l1_id_e+,q0_id_dq,q0_id_uq,q1_id_dq,q1_id_uq,weight,cHW
0,114.490093,42.362408,252.303723,31.771265,-2.354777,-1.332412,0.683512,3.075705,-1.327065,-1.488743,-2.031290,2.792608,-0.001031,0.000245,-0.003391,0.001526,230.996226,98.644673,978.317857,260.282905,100.998772,-2.743096,69.065507,1004.682902,141.278871,230.064283,-1.022365,-2.392193,0.161678,-4.823898,0.161678,-4.823898,23.279310,4.338564,360.783424,468.103452,183.860152,313.345669,1,0,0,1,0,1,1,0,0.001126,0.0
1,14.149076,8.927327,77.288809,56.531621,-0.431505,2.710642,2.792856,1.100597,3.263323,1.380969,-3.088486,2.372211,-0.001671,0.000148,0.002845,0.001935,185.172529,18.881695,849.767215,305.676230,85.765897,-1.055257,33.190396,1014.394249,5.221753,90.058769,3.141038,1.692260,1.882354,-5.460697,1.882354,-5.460697,15.577636,5.505707,793.330434,47.076272,242.640364,39.834522,0,1,1,0,0,1,1,0,0.001126,0.0
2,39.008558,9.447233,53.517911,45.112543,-0.772832,-1.588745,-1.351145,1.536884,0.136348,0.610233,-2.936717,4.250636,0.000140,-0.000070,0.002899,-0.006027,39.371721,11.261500,505.928428,1582.644088,59.978958,2.322270,17.785877,1788.256374,45.999397,15.001616,0.815913,-2.888029,-0.473885,-7.187353,-0.473885,-7.187353,14.389760,4.168169,204.231266,331.832043,128.762006,130.783442,1,0,0,1,0,1,1,0,0.001126,0.0
3,62.168410,31.144976,111.580677,52.043403,0.781485,0.513821,2.704831,-1.963006,2.335536,1.909460,-3.010168,3.131070,-0.000674,0.001113,-0.009419,-0.002325,324.262793,107.412830,1134.780921,596.993418,76.564319,-0.863603,22.242951,1642.755358,92.571564,121.002054,0.267664,-1.615349,0.426076,-6.141238,0.426076,-6.141238,18.262031,4.192541,1208.100598,120.814845,692.817691,92.349083,1,0,0,1,0,1,1,0,0.001126,0.0
4,51.878371,31.590252,83.984114,64.091713,0.331658,0.002331,-2.162397,2.160402,0.114070,0.821951,-2.420317,2.930326,0.000119,-0.000098,0.001038,0.001313,52.216255,42.875983,476.119309,602.049256,1.906682,-0.279127,32.128412,1066.996636,82.406723,84.096916,0.329327,1.960386,-0.707881,-5.350643,-0.707881,-5.350643,15.279108,0.124790,249.461615,239.727874,266.401816,138.428561,1,0,0,1,0,1,1,0,0.001126,0.0


In [29]:
all_dataset_df.shape

(11000000, 48)

In [30]:
all_dataset_df.to_csv('new_features_lo_all_data.csv', index=False)

# Npy train and test dataset creation

In [2]:
# all_data_df = all_dataset_df.copy()
# del all_dataset_df

all_data_df = pd.read_csv('new_features_lo_all_data.csv')

In [4]:
for i in range(12):
    look_idex = 1_000_000*i
    weight_value = all_data_df['weight'][look_idex:look_idex+1].copy().to_numpy()
    print(weight_value)
    cHW_value = all_data_df['cHW'][look_idex:look_idex+1].copy().to_numpy()
    print(cHW_value)

[0.00112598]
[0.]
[0.00114727]
[-0.1]
[0.0014416]
[-1.]
[0.0020291]
[-2.]
[0.00608081]
[-5.]
[0.0254231]
[-10.]
[0.00110791]
[0.1]
[0.00102545]
[1.]
[0.00109137]
[2.]
[0.00187389]
[5.]
[0.00371504]
[10.]
[]
[]


In [5]:
from sklearn.model_selection import train_test_split
SEED = 42


In [6]:
X_train, X_test = train_test_split(all_data_df, test_size=0.2, random_state=SEED, shuffle=True)


In [7]:
X_train.head(7)

,l0_pt,l1_pt,q0_pt,q1_pt,l0_phi,l1_phi,q0_phi,q1_phi,l0_eta,l1_eta,q0_eta,q1_eta,l0_m,l1_m,q0_m,q1_m,l0_e,l1_e,q0_e,q1_e,met_et,met_phi,m_ll,m_qq,pt_ll,pt_qq,d_phi_ll,d_phi_qq,d_eta_ll,d_eta_qq,d_y_ll,d_y_qq,sqrtHT,MET_sig,m_l0q0,m_l0q1,m_l1q0,m_l1q1,l0_id_e-,l0_id_e+,l1_id_e-,l1_id_e+,q0_id_dq,q0_id_uq,q1_id_dq,q1_id_uq,weight,cHW
1950072,47.215598,8.340444,77.538543,63.671204,-1.946468,-0.491243,0.518191,-2.616898,-0.220281,-2.238011,1.189458,-3.673168,-0.000093,0.000060,-0.001178,-0.011507,48.365772,39.539278,139.169551,1254.382085,41.410719,1.625978,54.068402,805.342270,48.884529,13.874866,-1.455225,3.135089,2.017730,4.862627,2.017730,4.862627,15.432968,2.683263,146.945783,300.593881,138.744720,54.004830,0,1,1,0,1,0,0,1,0.001147,-0.1
3342860,28.811619,21.919331,46.436089,23.466303,1.020461,1.650947,-2.937625,-2.316705,0.639616,1.286710,1.291608,-1.756191,0.000164,-0.000074,0.000404,0.000329,34.908854,42.710186,90.862919,69.965264,51.989651,-0.389988,22.729572,145.731251,48.278304,66.929450,-0.630485,-0.620920,-0.647094,3.047799,-0.647094,3.047799,13.138607,3.957014,71.395362,93.859635,47.823951,107.265877,0,1,1,0,1,0,0,1,0.002029,-2.0
14241,18.963905,12.853339,155.764835,67.831308,-0.250227,-1.688073,-2.564213,-0.143819,1.040321,0.897419,-1.257415,2.549562,0.000233,-0.000111,-0.001656,-0.002423,30.185476,18.385956,296.008400,436.820434,121.463280,1.177809,20.684833,701.222050,24.278812,113.990078,1.437845,-2.420393,0.142902,-3.806977,0.142902,-3.806977,19.413312,6.256701,183.549711,59.537948,122.229049,68.339541,0,1,1,0,0,1,1,0,0.001126,0.0
10112756,145.534621,129.940547,465.514223,271.038407,0.163323,-0.031062,2.629758,-1.641287,-1.420567,-1.530120,-1.037389,1.506025,0.001665,-0.000680,-0.002209,-0.001590,318.796822,314.146997,739.287352,641.082170,153.168776,0.149006,30.651309,1312.631192,274.179245,427.054041,0.194385,-2.012140,0.109553,-2.543414,0.109553,-2.543414,34.134976,4.487151,501.334581,869.826907,493.181178,859.007872,1,0,0,1,0,1,1,0,0.003715,10.0
5485587,27.554667,14.938415,184.101965,79.578256,-0.856351,-1.470337,0.754995,-2.243717,-0.422659,-1.748893,-0.325748,-1.682127,0.000069,0.000262,0.000470,-0.001548,30.052712,44.234145,193.956343,221.345224,107.797994,-2.876410,31.415021,299.418089,40.685410,105.942376,0.613986,2.998712,1.326234,1.356379,1.326234,1.356379,20.346285,5.298166,102.979165,86.882956,124.197286,26.107173,1,0,0,1,1,0,0,1,0.025423,-10.0
2288732,37.079696,16.729881,108.191444,72.991990,1.574572,-2.928279,0.756685,-1.282530,-0.498055,-2.039058,-3.240825,2.233500,-0.000099,0.000316,-0.005815,0.002193,41.774523,65.359650,1384.522566,344.508490,91.150114,-2.713587,57.336367,1374.969617,37.372736,99.511346,-1.780335,2.039215,1.541004,-5.474325,1.541004,-5.474325,18.059433,5.047230,238.901222,216.645000,98.300620,296.246676,1,0,0,1,0,1,1,0,0.001442,-1.0
3871865,45.617555,25.640466,58.360828,39.210620,-2.224349,-2.959703,1.565785,-0.170553,1.573543,1.467181,1.659161,-3.599806,0.001091,0.000201,0.000655,-0.000560,114.751415,58.557285,158.893162,717.916321,17.727336,-0.658636,24.854341,663.911964,66.882074,64.724608,0.735354,1.736338,0.106362,5.258967,0.106362,5.258967,13.658580,1.297890,97.916156,563.369218,60.035200,401.799153,0,1,1,0,1,0,0,1,0.002029,-2.0


In [8]:
X_test.head(7)

,l0_pt,l1_pt,q0_pt,q1_pt,l0_phi,l1_phi,q0_phi,q1_phi,l0_eta,l1_eta,q0_eta,q1_eta,l0_m,l1_m,q0_m,q1_m,l0_e,l1_e,q0_e,q1_e,met_et,met_phi,m_ll,m_qq,pt_ll,pt_qq,d_phi_ll,d_phi_qq,d_eta_ll,d_eta_qq,d_y_ll,d_y_qq,sqrtHT,MET_sig,m_l0q0,m_l0q1,m_l1q0,m_l1q1,l0_id_e-,l0_id_e+,l1_id_e-,l1_id_e+,q0_id_dq,q0_id_uq,q1_id_dq,q1_id_uq,weight,cHW
3967303,34.936882,9.803496,289.264064,156.967437,0.438533,3.128132,-2.757393,1.309795,2.600561,2.218967,-0.844580,1.382436,-0.001003,-0.000176,-0.000423,-0.001979,236.619259,45.618345,398.712650,332.422774,213.911501,-0.275533,36.765557,693.187644,26.466519,231.739151,-2.689599,2.215997,0.381594,-2.227016,0.381594,-2.227016,26.549640,8.057040,580.778803,114.458026,235.794469,70.518628,0,1,1,0,0,1,1,0,0.002029,-2.0
5946179,435.235697,65.063561,691.408181,668.089983,-0.627135,-0.475233,-2.824873,1.401754,-0.182713,-0.391366,2.059601,1.479025,-0.001426,0.000315,-0.020327,0.011413,442.520888,70.110299,2755.381639,1542.127330,207.436598,-0.866269,43.468316,1230.973517,499.647065,702.183725,-0.151902,2.056559,0.208653,0.580576,0.208653,0.580576,45.466845,4.562371,1793.960088,1358.058207,767.392239,561.291204,0,1,1,0,1,0,0,1,0.025423,-10.0
6910558,18.209970,16.341243,81.690912,38.212562,-1.636960,-1.805120,1.485921,3.004190,-0.846936,-1.028066,2.564542,-1.093563,0.000088,-0.000159,0.000584,0.000085,25.140829,25.764852,533.917693,63.431167,63.669628,-0.971277,4.264335,347.610869,34.429513,91.985850,0.168161,-1.518269,0.181130,3.658105,0.181130,3.658105,14.769032,4.311022,219.345754,39.157372,226.214836,33.626000,1,0,0,1,1,0,0,1,0.001108,0.1
3414332,26.409473,22.663260,123.569833,62.208535,0.272002,-3.068100,-1.071565,1.648530,1.360736,1.173434,0.390833,4.409743,0.000261,0.000048,0.000767,0.003341,54.872725,40.140940,133.128252,2558.650956,70.369042,2.518978,48.904498,664.740560,6.127154,71.489852,-2.943084,-2.720095,0.187302,-4.018910,0.187302,-4.018910,17.470551,4.027866,91.513579,184.660263,98.583549,189.497967,0,1,1,0,0,1,1,0,0.002029,-2.0
5840458,229.354544,107.835542,291.046281,62.368915,-1.691108,-1.845095,1.369399,1.474807,0.495262,0.209730,0.977914,-2.099809,0.000640,-0.001117,0.002147,-0.001219,258.062752,110.215910,441.662650,258.428369,17.422984,-2.013317,51.141421,598.998063,336.321058,353.130006,0.153987,-0.105408,0.285532,3.077723,0.285532,3.077723,26.608801,0.654783,531.438631,470.440975,380.556521,285.711331,0,1,1,0,1,0,0,1,0.025423,-10.0
4436076,79.016414,39.060340,624.729456,62.228088,-1.098391,-1.663782,1.870356,2.632834,0.554369,-0.034601,-0.342126,0.014893,0.000188,0.000187,0.002173,0.000296,91.472433,39.083724,661.649764,62.234988,557.565293,-1.191639,45.415403,162.896655,113.936387,671.106087,0.565391,-0.762478,0.588970,-0.357018,0.588970,-0.357018,36.913407,15.104683,488.249603,139.548228,310.197418,82.648189,0,1,1,0,0,1,1,0,0.006081,-5.0
7461995,83.279384,7.613732,93.390104,47.423699,-1.674648,-0.570254,1.863846,-0.619377,-0.470686,0.751479,1.125875,-4.477297,0.000160,0.000046,-0.001229,-0.009866,92.676046,9.866644,159.103134,2086.829563,26.772117,1.930038,42.058311,1099.327148,86.969382,62.962476,-1.104394,2.483223,-1.222165,5.603172,-1.222165,5.603172,16.077283,1.665214,233.047633,461.777977,51.026623,258.154449,1,0,0,1,1,0,0,1,0.001025,1.0


In [9]:
cHW_values = sorted(X_train['cHW'].unique())
cHW_values

[-10.0, -5.0, -2.0, -1.0, -0.1, 0.0, 0.1, 1.0, 2.0, 5.0, 10.0]

In [10]:
for cHW in cHW_values:
    print(cHW)
    print(X_train[X_train['cHW'] == cHW].shape[0])
    print(X_test[X_test['cHW'] == cHW].shape[0])

-10.0
800607
199393
-5.0
800577
199423
-2.0
799718
200282
-1.0
799893
200107
-0.1
799327
200673
0.0
799961
200039
0.1
799503
200497
1.0
799986
200014
2.0
799953
200047
5.0
799811
200189
10.0
800664
199336


In [11]:
deneme_df = X_train[X_train['cHW'] == 5.0].copy()
deneme_df.head(5)

,l0_pt,l1_pt,q0_pt,q1_pt,l0_phi,l1_phi,q0_phi,q1_phi,l0_eta,l1_eta,q0_eta,q1_eta,l0_m,l1_m,q0_m,q1_m,l0_e,l1_e,q0_e,q1_e,met_et,met_phi,m_ll,m_qq,pt_ll,pt_qq,d_phi_ll,d_phi_qq,d_eta_ll,d_eta_qq,d_y_ll,d_y_qq,sqrtHT,MET_sig,m_l0q0,m_l0q1,m_l1q0,m_l1q1,l0_id_e-,l0_id_e+,l1_id_e-,l1_id_e+,q0_id_dq,q0_id_uq,q1_id_dq,q1_id_uq,weight,cHW
9082688,32.527951,31.621283,63.809833,52.006715,-2.539044,-1.289596,0.955668,1.010013,-0.818492,-1.621180,-2.616002,3.661961,0.000205,0.000020,0.001825,0.010226,44.045701,83.112114,438.820795,1013.176669,66.328225,-2.346783,45.896435,1327.092204,52.035715,115.774238,-1.249448,-0.054345,0.802687,-6.277963,0.802687,-6.277963,15.693757,4.226408,129.477645,390.457801,93.395520,571.094179,0,1,1,0,0,1,1,0,0.001874,5.0
9007831,115.053271,10.776550,276.888939,76.618191,0.951563,1.502219,-2.731956,-1.171678,0.003871,0.885591,1.836197,-0.301626,-0.000890,-0.000138,0.002539,-0.000234,115.054133,15.285922,890.483393,80.129984,174.337727,0.453837,37.343822,426.586831,124.364744,288.069641,-0.550656,-1.560278,-0.881720,2.137823,-0.881720,2.137823,25.567062,6.818841,508.663533,166.467236,107.795424,66.576578,1,0,0,1,1,0,0,1,0.001874,5.0
9358392,58.880768,20.379892,176.076368,123.118092,1.611977,2.062538,-1.943880,0.971684,-1.100045,-0.489081,-0.263438,-0.668111,-0.000134,-0.000113,-0.001689,0.001120,98.247233,22.866304,182.221601,151.633839,16.021017,-1.132654,26.486702,298.678221,77.735077,62.509099,-0.450561,-2.915564,-0.610965,0.404673,-0.610965,0.404673,19.861423,0.806640,217.728513,65.157530,109.619564,52.743076,1,0,0,1,0,1,1,0,0.001874,5.0
9016328,54.565072,10.911173,84.993101,38.686812,1.024804,-1.942408,-2.696823,-1.505575,-1.935355,-0.626703,4.292303,-1.460285,-0.001551,-0.000057,0.008815,0.000185,192.911386,13.124946,3108.553479,87.806630,63.098105,0.660416,59.473579,1016.535862,43.860250,105.626665,2.967211,-1.191248,-1.308652,5.752588,-1.308652,5.752588,15.882514,3.972803,1535.269678,90.359722,354.375156,19.747722,0,1,1,0,1,0,0,1,0.001874,5.0
9937495,46.235024,36.953550,231.839528,24.814769,2.326568,2.423556,-0.620172,-1.000064,0.400943,1.888840,0.157983,3.427348,-0.000021,-0.001308,0.000902,-0.001775,50.001342,124.957361,234.738740,382.486504,172.781009,2.540875,67.452934,375.242890,83.091994,255.051167,-0.096989,0.379892,-1.487897,-3.269365,-1.487897,-3.269365,22.641199,7.631266,207.621277,161.158395,258.721338,78.926762,0,1,1,0,0,1,1,0,0.001874,5.0


In [12]:
label = deneme_df['weight'].copy()
np.unique(label.to_numpy())
label.shape

(799811,)

In [13]:
feautures = deneme_df.drop(['weight', 'cHW'], axis=1).copy()
feautures.head(5)

,l0_pt,l1_pt,q0_pt,q1_pt,l0_phi,l1_phi,q0_phi,q1_phi,l0_eta,l1_eta,q0_eta,q1_eta,l0_m,l1_m,q0_m,q1_m,l0_e,l1_e,q0_e,q1_e,met_et,met_phi,m_ll,m_qq,pt_ll,pt_qq,d_phi_ll,d_phi_qq,d_eta_ll,d_eta_qq,d_y_ll,d_y_qq,sqrtHT,MET_sig,m_l0q0,m_l0q1,m_l1q0,m_l1q1,l0_id_e-,l0_id_e+,l1_id_e-,l1_id_e+,q0_id_dq,q0_id_uq,q1_id_dq,q1_id_uq
9082688,32.527951,31.621283,63.809833,52.006715,-2.539044,-1.289596,0.955668,1.010013,-0.818492,-1.621180,-2.616002,3.661961,0.000205,0.000020,0.001825,0.010226,44.045701,83.112114,438.820795,1013.176669,66.328225,-2.346783,45.896435,1327.092204,52.035715,115.774238,-1.249448,-0.054345,0.802687,-6.277963,0.802687,-6.277963,15.693757,4.226408,129.477645,390.457801,93.395520,571.094179,0,1,1,0,0,1,1,0
9007831,115.053271,10.776550,276.888939,76.618191,0.951563,1.502219,-2.731956,-1.171678,0.003871,0.885591,1.836197,-0.301626,-0.000890,-0.000138,0.002539,-0.000234,115.054133,15.285922,890.483393,80.129984,174.337727,0.453837,37.343822,426.586831,124.364744,288.069641,-0.550656,-1.560278,-0.881720,2.137823,-0.881720,2.137823,25.567062,6.818841,508.663533,166.467236,107.795424,66.576578,1,0,0,1,1,0,0,1
9358392,58.880768,20.379892,176.076368,123.118092,1.611977,2.062538,-1.943880,0.971684,-1.100045,-0.489081,-0.263438,-0.668111,-0.000134,-0.000113,-0.001689,0.001120,98.247233,22.866304,182.221601,151.633839,16.021017,-1.132654,26.486702,298.678221,77.735077,62.509099,-0.450561,-2.915564,-0.610965,0.404673,-0.610965,0.404673,19.861423,0.806640,217.728513,65.157530,109.619564,52.743076,1,0,0,1,0,1,1,0
9016328,54.565072,10.911173,84.993101,38.686812,1.024804,-1.942408,-2.696823,-1.505575,-1.935355,-0.626703,4.292303,-1.460285,-0.001551,-0.000057,0.008815,0.000185,192.911386,13.124946,3108.553479,87.806630,63.098105,0.660416,59.473579,1016.535862,43.860250,105.626665,2.967211,-1.191248,-1.308652,5.752588,-1.308652,5.752588,15.882514,3.972803,1535.269678,90.359722,354.375156,19.747722,0,1,1,0,1,0,0,1
9937495,46.235024,36.953550,231.839528,24.814769,2.326568,2.423556,-0.620172,-1.000064,0.400943,1.888840,0.157983,3.427348,-0.000021,-0.001308,0.000902,-0.001775,50.001342,124.957361,234.738740,382.486504,172.781009,2.540875,67.452934,375.242890,83.091994,255.051167,-0.096989,0.379892,-1.487897,-3.269365,-1.487897,-3.269365,22.641199,7.631266,207.621277,161.158395,258.721338,78.926762,0,1,1,0,0,1,1,0


In [14]:
feautures.shape

(799811, 46)

In [15]:
del label, feautures, deneme_df

# I was told to only train for negative values for the moment.

In [16]:
for cHW in cHW_values:
    if float(cHW) < 0.0:
        print(f'We are working on cHW = {cHW}')
        train_subset_df = X_train[X_train['cHW'] == cHW].copy()
        train_subset_feautures = train_subset_df.drop(['weight', 'cHW'], axis=1).copy()
        # labels = train_subset_df['weight'].copy()
        np_save_path = f'train_bsm_{cHW}.npy'
        train_features_np = train_subset_feautures.to_numpy()
        np.save(np_save_path, train_features_np)
        del train_subset_df, train_subset_feautures, train_features_np
    

We are working on cHW = -10.0
We are working on cHW = -5.0
We are working on cHW = -2.0
We are working on cHW = -1.0
We are working on cHW = -0.1


In [17]:
for cHW in cHW_values:
    if float(cHW) < 0.0:
        print(f'We are working on cHW = {cHW}')
        test_subset_df = X_test[X_test['cHW'] == cHW].copy()
        test_subset_feautures = test_subset_df.drop(['weight', 'cHW'], axis=1).copy()
        # labels = test_subset_df['weight'].copy()
        np_save_path = f'test_bsm_{cHW}.npy'
        test_features_np = test_subset_feautures.to_numpy()
        np.save(np_save_path, test_features_np)
        del test_subset_df, test_subset_feautures, test_features_np
    

We are working on cHW = -10.0
We are working on cHW = -5.0
We are working on cHW = -2.0
We are working on cHW = -1.0
We are working on cHW = -0.1
We are working on cHW = 0.0
We are working on cHW = 0.1
We are working on cHW = 1.0
We are working on cHW = 2.0
We are working on cHW = 5.0
We are working on cHW = 10.0


In [18]:
def find_files(search_dir=".", file_name_to_be_searched=None):
    search_pattern = os.path.join(search_dir, "**", file_name_to_be_searched)
    return glob.glob(search_pattern, recursive=True)

def get_cHW(path):
    return float(path.split("/")[-1].split("_")[-1][:-4])

# old_cHW_dic = {0.0: 0.00112598, -0.1: 0.001147323, -1.0: 0.0014416, -2.0: 0.0020291, -5.0: 0.00608081, -10.0: 0.025434, 0.1: 0.001107908, 1.0: 0.001025449, 2.0: 0.001091368,
# 5.0: 0.00187389, 10.0: 0.00371504}

cHW_dic = {0.0: 0.00112598,
 -0.1: 0.001147274,
 -1.0: 0.0014416,
 -2.0: 0.0020291,
 -5.0: 0.00608081,
 -10.0: 0.0254231,
 0.1: 0.001107908,
 1.0: 0.001025449,
 2.0: 0.001091368,
 5.0: 0.00187389,
 10.0: 0.00371504,}

reverse_dic = {v: k for k, v in cHW_dic.items()}

diff_corss_section_ratio_dic = {
1.0: 0.0,1.01891153: -0.1,
1.28030693: -1.0,
1.80207464: -2.0,
5.40046004: -5.0,
22.5786426: -10.0,
0.98394998: 0.1,
0.91071689: 1.0,
0.96926056: 2.0,
1.66423027: 5.0,
3.29938365: 10.0}

# old_diff_corss_section_ratio_dic = {1.0: 0.0, 1.01895504: -0.1, 1.28030693: -1.0, 1.80207464: -2.0, 5.40046004: -5.0, 22.58832306: -10.0, 0.98394998: 0.1, 0.91071689: 1.0, 
# 0.96926056: 2.0, 1.66423027: 5.0, 3.29938365: 10.0}
# old_diff_corss_section_ratio_dic

In [6]:
train_npy_paths = find_files(search_dir=f"{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/npy_files", file_name_to_be_searched="train*.npy")
train_npy_paths = sorted(train_npy_paths)
# train_npy_paths.remove(f'{LOCAL_PRE_PATH}/datasets/all_data/lo_root_files/new_features/csv_files/train_bsm_0.0.npy')
train_npy_paths

['/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/npy_files/train_bsm_-0.1.npy',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/npy_files/train_bsm_-1.0.npy',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/npy_files/train_bsm_-10.0.npy',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/npy_files/train_bsm_-2.0.npy',
 '/Users/atakanazakli/SFU_Thesis_Work/datasets/all_data/lo_root_files/new_features/npy_files/train_bsm_-5.0.npy']

In [23]:
from sklearn.utils import shuffle

In [24]:
# Load and concatenate all .npy files
all_data = []
all_weight = []
for path in train_npy_paths:
    data = np.load(path)
    all_data.append(data)
    label_value = cHW_dic[get_cHW(path)]
    weight = np.full(len(data), label_value)
    all_weight.append(weight)

# Convert the list of arrays into a single np.ndarray
features = np.concatenate(all_data, axis=0)
all_weight_array = np.concatenate(all_weight, axis=0)

labels = all_weight_array / 0.00112598


features, labels = shuffle(features, labels, random_state=42)

# Print the shapes of the concatenated arrays
print('features.spahe: ', features.shape)
print('labels.spahe: ', labels.shape)

features.spahe:  (4000122, 46)
labels.spahe:  (4000122,)


In [25]:
np.save("train_features.npy", features)

In [26]:
np.save("train_labels.npy", labels)